In [1]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file= 'banglaIsolated_clean.pickle'

with open(pickle_file, 'rb') as f:
    save=pickle.load(f)
    train_dataset=save['train_dataset']
    train_labels=save['train_labels']
    valid_dataset=save['valid_dataset_clean']
    valid_labels=save['valid_labels']
    test_dataset=save['test_dataset_clean']
    test_labels=save['test_labels']
    del save #hint to help gc to free up memory 
    print('training set', train_dataset.shape,train_labels.shape)
    print('validation set', valid_dataset.shape, valid_labels.shape)
    print('test set', test_dataset.shape,test_labels.shape)

training set (139000, 28, 28) (139000,)
validation set (10000, 28, 28) (10000,)
test set (10000, 28, 28) (10000,)


In [3]:
image_size=28
num_labels=10
num_channels=1 

def reformat(dataset, labels):
    dataset=dataset.reshape((-1,image_size,image_size,num_channels)).astype(np.float32)
    # Map 1 to [0.0,1.0,0.0....], 2 to [0.0,0.0,1.0.....]
    labels=(np.arange(num_labels) ==labels[:,None]).astype(np.float32)
    return dataset,labels
train_dataset, train_labels= reformat(train_dataset, train_labels)
valid_dataset, valid_labels=reformat(valid_dataset, valid_labels)
test_dataset, test_labels =reformat(test_dataset, test_labels)
print( 'training set', train_dataset.shape,train_labels.shape)
print('validation set', valid_dataset.shape,valid_labels.shape)
print('test set', test_dataset.shape,test_labels.shape)

training set (139000, 28, 28, 1) (139000, 10)
validation set (10000, 28, 28, 1) (10000, 10)
test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return(100.0*np.sum(np.argmax(predictions, 1)==np.argmax(labels,1))/ predictions.shape[0])

In [6]:
batch_size=16
patch_size=5
depth=16
num_hidden=64

graph=tf.Graph()

with graph.as_default():
    
    #input data
    tf_train_dataset=tf.placeholder(tf.float32, 
                                    shape=(batch_size,image_size,image_size,num_channels))
    tf_train_labels=tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    #variables
    layer1_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,num_channels,depth],stddev=0.1))
    layer1_biases=tf.Variable(tf.zeros([depth]))
    
    layer2_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,depth,depth],stddev=0.1))
    layer2_biases=tf.Variable(tf.constant(1.0,shape=[depth]))
    
    
    layer3_weights=tf.Variable(tf.truncated_normal
                               ([image_size//4*image_size//4*depth,num_hidden],stddev=0.1))
    layer3_biases=tf.Variable(tf.constant(1.0,shape=[num_hidden]))
    
    
    layer4_weights=tf.Variable(tf.truncated_normal
                               ([num_hidden,num_labels],stddev=0.1))
    layer4_biases=tf.Variable(tf.constant(1.0,shape=[num_labels]))
    
    #Model
    def model(data):
        conv=tf.nn.conv2d(data,layer1_weights,[1,2,2,1], padding='SAME')
        hidden=tf.nn.relu(conv+layer1_biases)
        conv=tf.nn.conv2d(hidden,layer2_weights,[1,2,2,1], padding='SAME')
        hidden=tf.nn.relu(conv+layer2_biases)
        shape=hidden.get_shape().as_list()
        reshape=tf.reshape(hidden,[shape[0],shape[1]*shape[2]*shape[3]])
        hidden=tf.nn.relu(tf.matmul(reshape, layer3_weights)+layer3_biases)
        return tf.matmul(hidden, layer4_weights)+ layer4_biases
    
    #training computation
    
    logits=model(tf_train_dataset)
    loss=tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    #optimizer
    
    optimizer=tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    #Predictions for the training , validation, and test data
    
    train_prediction= tf.nn.softmax(logits)
    valid_prediction= tf.nn.softmax(model(tf_valid_dataset))
    test_prediction= tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps=10001

with tf.Session(graph=graph)as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset=(step*batch_size)% (train_labels.shape[0]-batch_size)
        batch_data=train_dataset[offset:(offset+batch_size),:,:,:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        feed_dict={tf_train_dataset : batch_data,tf_train_labels: batch_labels}
        _, l, predictions =session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if(step%50==0):
            print('Minibatch loss a step %d: %f' %(step,l))
            print('Minibatch Accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%'%accuracy(valid_prediction.eval(),valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    

Initialized
Minibatch loss a step 0: 0.534299
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 50: 0.109666
Minibatch Accuracy: 6.2%
Validation accuracy: 1.2%
Minibatch loss a step 100: 0.156741
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 150: 18.190140
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 200: 10.557732
Minibatch Accuracy: 6.2%
Validation accuracy: 89.3%
Minibatch loss a step 250: 19.511017
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 300: 26.873718
Minibatch Accuracy: 93.8%
Validation accuracy: 1.2%
Minibatch loss a step 350: 195.372070
Minibatch Accuracy: 93.8%
Validation accuracy: 1.2%
Minibatch loss a step 400: 1232.872070
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 450: 6471.355469
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 500: 48003.046875
Minibatch Accuracy: 6.2%
Validation accuracy: 1.2%
Minibatch loss 

# convulation using maxpooling operation(nn.max_pool()) of stride of 2 and kernel size 2.

In [5]:
batch_size=16
patch_size=5
depth=16
num_hidden=64

graph=tf.Graph()

with graph.as_default():
    
    #input data
    tf_train_dataset=tf.placeholder(tf.float32, 
                                    shape=(batch_size,image_size,image_size,num_channels))
    tf_train_labels=tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    #variables
    layer1_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,num_channels,depth],stddev=0.1))
    layer1_biases=tf.Variable(tf.zeros([depth]))
    
    layer2_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,depth,depth],stddev=0.1))
    layer2_biases=tf.Variable(tf.constant(1.0,shape=[depth]))
    
    
    layer3_weights=tf.Variable(tf.truncated_normal
                               ([image_size//4*image_size//4*depth,num_hidden],stddev=0.1))
    layer3_biases=tf.Variable(tf.constant(1.0,shape=[num_hidden]))
    
    
    layer4_weights=tf.Variable(tf.truncated_normal
                               ([num_hidden,num_labels],stddev=0.1))
    layer4_biases=tf.Variable(tf.constant(1.0,shape=[num_labels]))
    
    #Model
    def model(data):
        #1st conv layer with pooling
        conv_1=tf.nn.conv2d(data,layer1_weights,[1,1,1,1], padding='SAME')
        hidden_1=tf.nn.relu(conv_1+layer1_biases)
        pool_1=tf.nn.max_pool(hidden_1,[1,2,2,1],[1,2,2,1], padding='SAME')
        
         #2nd conv layer with pooling
        conv_2=tf.nn.conv2d(pool_1,layer2_weights,[1,1,1,1], padding='SAME')
        hidden_2=tf.nn.relu(conv_2+layer2_biases)
        pool_2=tf.nn.max_pool(hidden_2,[1,2,2,1],[1,2,2,1], padding='SAME')
        
         #fully connected layer
        shape=pool_2.get_shape().as_list()
        reshape=tf.reshape(pool_2,[shape[0],shape[1]*shape[2]*shape[3]])
        hidden=tf.nn.relu(tf.matmul(reshape, layer3_weights)+layer3_biases)
        return tf.matmul(hidden, layer4_weights)+ layer4_biases
    
    #training computation
    
    logits=model(tf_train_dataset)
    loss=tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    #optimizer
    
    optimizer=tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    #Predictions for the training , validation, and test data
    
    train_prediction= tf.nn.softmax(logits)
    valid_prediction= tf.nn.softmax(model(tf_valid_dataset))
    test_prediction= tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps=10001

with tf.Session(graph=graph)as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset=(step*batch_size)% (train_labels.shape[0]-batch_size)
        batch_data=train_dataset[offset:(offset+batch_size),:,:,:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        feed_dict={tf_train_dataset : batch_data,tf_train_labels: batch_labels}
        _, l, predictions =session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if(step%50==0):
            print('Minibatch loss a step %d: %f' %(step,l))
            print('Minibatch Accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%'%accuracy(valid_prediction.eval(),valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    

Initialized
Minibatch loss a step 0: 0.276357
Minibatch Accuracy: 0.0%
Validation accuracy: 1.1%
Minibatch loss a step 50: 0.104936
Minibatch Accuracy: 6.2%
Validation accuracy: 1.2%
Minibatch loss a step 100: 0.133598
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 150: 8.187197
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 200: 23.321716
Minibatch Accuracy: 81.2%
Validation accuracy: 1.2%
Minibatch loss a step 250: 59.679962
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 300: 26.604858
Minibatch Accuracy: 6.2%
Validation accuracy: 89.3%
Minibatch loss a step 350: 199.094238
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 400: 2530.550781
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 450: 11834.210938
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 500: 51271.875000
Minibatch Accuracy: 6.2%
Validation accuracy: 1.2%
Minibatch loss a

# conv with dropout

In [5]:
image_size=28
#create image size function based on input, filter size, padding and stride
#2 convolutions only with 2 pooling
def output_size_pool(input_size, conv_filter_size,pool_filter_size, padding, conv_stride, pool_stride):
    if padding =='same':
        padding=-1.00
    elif padding=='valid':
        padding=0.00
    else:
        return None
    #after convolution 1
    output_1=(((input_size-conv_filter_size-2*padding)/ conv_stride)+1.00)
    #after pool 1
    output_2=(((output_1-pool_filter_size-2*padding)/pool_stride)+1.00)
    #After convolution 2
    output_3=(((output_2-conv_filter_size-2*padding)/conv_stride)+1.00)
    #after pool 2
    output_4=(((output_3-pool_filter_size-2*padding)/pool_stride)+1.00)
    return int(output_4)
final_image_size=output_size_pool(
    input_size=image_size,conv_filter_size=5,pool_filter_size=2,
    padding='valid', conv_stride=1, pool_stride=2)
print(final_image_size)
    

4


In [6]:
batch_size=16
patch_size=5
depth=32
num_hidden=64
beta=0.001

graph=tf.Graph()

with graph.as_default():
    
    #input data
    tf_train_dataset=tf.placeholder(tf.float32, 
                                    shape=(batch_size,image_size,image_size,num_channels))
    tf_train_labels=tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    #variables
    layer1_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,num_channels,depth],stddev=0.1))
    layer1_biases=tf.Variable(tf.zeros([depth]))
    
    layer2_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,depth,depth],stddev=0.1))
    layer2_biases=tf.Variable(tf.constant(1.0,shape=[depth]))
    
    final_image_size=output_size_pool(input_size=image_size,
                                      conv_filter_size=5,pool_filter_size=2,padding='valid',
                                      conv_stride=1,pool_stride=2)
    
    
    layer3_weights=tf.Variable(tf.truncated_normal
                               ([final_image_size*final_image_size*depth,num_hidden],stddev=0.1))
    layer3_biases=tf.Variable(tf.constant(1.0,shape=[num_hidden]))
    
    
    layer4_weights=tf.Variable(tf.truncated_normal
                               ([num_hidden,num_hidden],stddev=0.1))
    layer4_biases=tf.Variable(tf.constant(1.0,shape=[num_hidden]))
    
    
    
    layer5_weights=tf.Variable(tf.truncated_normal
                               ([num_hidden,num_labels],stddev=0.1))
    layer5_biases=tf.Variable(tf.constant(1.0,shape=[num_labels]))
    saver=tf.train.Saver()
    #Model
    def model(data):
        #1st conv layer with pooling
        conv_1=tf.nn.conv2d(data,layer1_weights,[1,1,1,1], padding='VALID')
        hidden_1=tf.nn.relu(conv_1+layer1_biases)
        pool_1=tf.nn.avg_pool(hidden_1,[1,2,2,1],[1,2,2,1], padding='VALID')
        
         #2nd conv layer with pooling
        conv_2=tf.nn.conv2d(pool_1,layer2_weights,[1,1,1,1], padding='VALID')
        hidden_2=tf.nn.relu(conv_2+layer2_biases)
        pool_2=tf.nn.avg_pool(hidden_2,[1,2,2,1],[1,2,2,1], padding='VALID')
        
         #1st fully connected layer
        shape=pool_2.get_shape().as_list()
        reshape=tf.reshape(pool_2,[shape[0],shape[1]*shape[2]*shape[3]])
        hidden=tf.nn.relu(tf.matmul(reshape, layer3_weights)+layer3_biases)
        keep_prob=0.5
        hidden_drop=tf.nn.dropout(hidden,keep_prob)
        
        #2nd fully connected layer
        hidden_2=tf.nn.relu(tf.matmul(hidden_drop, layer4_weights)+layer4_biases)
        hidden_2_drop=tf.nn.dropout(hidden_2,keep_prob)
        
        return tf.matmul(hidden_2_drop, layer5_weights)+ layer5_biases
    
    #training computation
    
    logits=model(tf_train_dataset)
    loss=tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    #optimizer
    global_step=tf.Variable(0)
    start_learning_rate=0.05
    learning_rate=tf.train.exponential_decay(start_learning_rate, global_step,100000,0.96,
                                             staircase=True)
    
    optimizer=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    #Predictions for the training , validation, and test data
    
    train_prediction= tf.nn.softmax(logits)
    valid_prediction= tf.nn.softmax(model(tf_valid_dataset))
    test_prediction= tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps=50000

with tf.Session(graph=graph)as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset=(step*batch_size)% (train_labels.shape[0]-batch_size)
        batch_data=train_dataset[offset:(offset+batch_size),:,:,:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        feed_dict={tf_train_dataset : batch_data,tf_train_labels: batch_labels}
        _, l, predictions =session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if(step%50==0):
            print('Minibatch loss a step %d: %f' %(step,l))
            print('Minibatch Accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%'%accuracy(valid_prediction.eval(),valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    
    saver.save(session, 'Bangla_CNN')
 

Initialized
Minibatch loss a step 0: 0.482663
Minibatch Accuracy: 6.2%
Validation accuracy: 5.9%
Minibatch loss a step 50: 0.072339
Minibatch Accuracy: 6.2%
Validation accuracy: 1.1%
Minibatch loss a step 100: 0.144300
Minibatch Accuracy: 0.0%
Validation accuracy: 1.1%
Minibatch loss a step 150: 0.555532
Minibatch Accuracy: 6.2%
Validation accuracy: 1.2%
Minibatch loss a step 200: 0.535046
Minibatch Accuracy: 6.2%
Validation accuracy: 1.2%
Minibatch loss a step 250: 0.276507
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 300: 0.288271
Minibatch Accuracy: 0.0%
Validation accuracy: 1.3%
Minibatch loss a step 350: 0.149537
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 400: 19.541321
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 450: 130.468201
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 500: 279.428955
Minibatch Accuracy: 12.5%
Validation accuracy: 1.2%
Minibatch loss a step 550: 4